In [106]:
#output line initial module
class outputFunction():
    def __init__(self,constant,Actualwind,Humidity,WindSpeed,WindDirection,Temperature,Pressure):
        self.coeff = [constant,Actualwind,Humidity,WindSpeed,WindDirection,Temperature,Pressure]
        self.root = [1,1,1,1,1,1,1]

In [107]:
# 数据标准模型
class data():
    def __init__(self,actualWindSpeed,Humidity,WindSpeed,WindDirection,Temperature,Pressure,Power,average,standardDeviation):
        self.GeneratedPower = Power
        self.ActualwindSpeed = (actualWindSpeed-average[0])/standardDeviation[0]
        self.Humidity = (Humidity-average[1])/standardDeviation[1]
        self.WindSpeed = (WindSpeed-average[2])/standardDeviation[2]
        self.WindDirection = (WindDirection-average[3])/standardDeviation[3]
        self.Temperature = (Temperature-average[4])/standardDeviation[4]
        self.Pressure = (Pressure-average[5])/standardDeviation[5]



In [120]:
#read data
import pandas as pd
trainingSet = pd.read_excel("2018.xlsx") 
trainingSet = trainingSet.to_dict('records')



In [109]:
#标准化数据计算
average = [0,0,0,0,0,0]
size = len(trainingSet)
for row in trainingSet:
    average[0] += row["Actual Wind Speed"] 
    average[1] += row["Humidity"]
    average[2] += row["Wind Speed"]
    average[3] += row["Wind Direction"]
    average[4] += row["Temperature"]
    average[5] += row["Pressure"]
for i in range(len(average)):
    average[i] = average[i]/size
standardDeviation = [0,0,0,0,0,0]
for row in trainingSet:
    standardDeviation[0] += (row["Actual Wind Speed"]-average[0])**2 
    standardDeviation[1] += (row["Humidity"]-average[1])**2 
    standardDeviation[2] += (row["Wind Speed"]-average[2])**2 
    standardDeviation[3] += (row["Wind Direction"]-average[3])**2 
    standardDeviation[4] += (row["Temperature"]-average[4])**2 
    standardDeviation[5] += (row["Pressure"]-average[5])**2 
for i in range(len(standardDeviation)):
    standardDeviation[i] = (standardDeviation[i]/(size-1))**0.5
    

In [110]:
#将数据放进class中转换成 标准化数据
dataSet = []
for row in trainingSet:
    dataSet.append(data(row["Actual Wind Speed"],row["Humidity"],row["Wind Speed"],row["Wind Direction"],row["Temperature"],row["Pressure"],row["Generated Power"],average,standardDeviation))
del trainingSet


In [111]:
global dataSet

In [115]:
#计算costFunction
def calculate(dataSet,lasthFunction):
    costFunction = 0
    for i in dataSet:
        #temp use to calculate the conv of each data
        temp = lasthFunction.coeff[0]+(lasthFunction.coeff[1]*i.ActualwindSpeed)**lasthFunction.root[1]+(lasthFunction.coeff[2]*i.Humidity)**lasthFunction.root[2]+(lasthFunction.coeff[3]*i.WindSpeed)**lasthFunction.root[3]+(lasthFunction.coeff[4]*i.Temperature)**lasthFunction.root[4]+(lasthFunction.coeff[5]*i.Pressure)**lasthFunction.root[5]
        temp = temp -i.GeneratedPower
        temp = temp ** 2
        costFunction = costFunction + temp
    costFunction = costFunction/(2*len(dataSet))
    return costFunction

In [116]:
#计算斜率
def derivative(locate,lasthFunction):
    total = 0
    for i in dataSet:
        variable = [1,i.ActualwindSpeed,i.Humidity,i.WindSpeed,i.WindDirection,i.Temperature,i.Pressure,i.GeneratedPower]
        temp = 0
        for i in range(len(variable)-1):
            temp = temp + variable[i]*lasthFunction.coeff[i]*lasthFunction.root[i]
        temp = temp - variable[-1] 
        temp = temp*variable[locate]*lasthFunction.root[locate]
        total = total + temp
    gradient = total/(len(dataSet))
    return gradient

    

In [ ]:
#initial
lasthFunction = outputFunction(1,1,1,1,1,1,1)
costFunction = calculate(dataSet,lasthFunction)
learningRate = 0.1
gradient = [0,0,0,0,0,0,0]
#gradient decent loop,while suatable stop
while(costFunction != 0):
    #temp = coeff of variable
    temp = [lasthFunction.coeff[0],lasthFunction.coeff[1],lasthFunction.coeff[2],lasthFunction.coeff[3],lasthFunction.coeff[4],lasthFunction.coeff[5],lasthFunction.coeff[6]]
    for i in range (len(temp)):
        gradient[i] = derivative(i,lasthFunction)
        temp[i]= temp[i] - learningRate*gradient[i]
    print(gradient)
    lasthFunction = outputFunction(temp[0],temp[1],temp[2],temp[3],temp[4],temp[5],temp[6])
    costFunction = calculate(dataSet,lasthFunction)



In [118]:
#final function after regression
lasthFunction.coeff


[16.73356790483466,
 11.602593431782172,
 1.4778504842742912,
 2.091999359729413,
 1.0196131332074578,
 -2.0391635433127577,
 -0.5772943894673654]

In [119]:
#verify
testSet = pd.read_excel("2019 .xlsx") 
testSet = testSet.values.tolist()
ems = []
predict = []
goal = []
total = 0
for i in testSet:
    temp = data(i[2],i[3],i[4],i[5],i[6],i[7],i[1],average,standardDeviation)
    ans = lasthFunction.coeff[0]+temp.ActualwindSpeed*lasthFunction.coeff[1]+temp.Humidity*lasthFunction.coeff[2]+temp.WindSpeed*lasthFunction.coeff[3]+temp.WindDirection*lasthFunction.coeff[4]+temp.Temperature*lasthFunction.coeff[5]+temp.Pressure*lasthFunction.coeff[6]
    predict.append(ans)
    accurancy = (ans-temp.GeneratedPower)/temp.GeneratedPower*100
    goal.append(temp.GeneratedPower)
    total = total + (ans - temp.GeneratedPower)**2
rmse = (total/len(testSet))** 0.5
print(1-(rmse/49.5))




0.9260664636309612
